# Applied Data Science Capstone
## Week 3 -> Segementing and Clustering Neighbourhoods in Toronto

# Part 1

In [2]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
import requests # library to handle requests
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from geopy.geocoders import Nominatim
from sklearn import preprocessing

### Downloading data from web saving in a dataframe

In [3]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=df[0]
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing the cells where Borough is not assigned

In [4]:
df=df[~(df['Borough']=='Not assigned')].reset_index(drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [5]:
df_cleaned=pd.DataFrame([]) # Dataframe to save post processed data

### Cleaning data to remove the Non assigned Boroughs and adding multiples neighbourhoods in a signle string

In [6]:
df_cleaned['Postcode']=df['Postcode'].unique()
df_cleaned.set_index('Postcode',inplace=True)
df_grouped=df.groupby('Postcode')
for (postcode,group) in df_grouped:
    df_cleaned.loc[postcode,'Borough']=     ', '.join(set(group.Borough))
    df_cleaned.loc[postcode,'Neighborhood']=', '.join(set(group.Neighborhood))
    if df_cleaned.loc[postcode,'Neighborhood']=='Not assigned':
        df_cleaned.loc[postcode,'Neighborhood']=df_cleaned.loc[postcode,'Borough']
df_cleaned.reset_index(drop=False, inplace=True)
df_cleaned.head(5)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [7]:
df_cleaned.shape

(103, 3)

# Part 2

### Merging the clearned data with the geospacial data

In [8]:
geospacial=pd.read_csv('http://cocl.us/Geospatial_data')
Merged_df=pd.merge(df_cleaned,geospacial,left_on='Postcode',right_on='Postal Code')
Merged_df.drop('Postal Code',inplace=True,axis=1)

In [9]:
Merged_df.head(5)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


# Part 3

## Exploring the clustering the neighborhoods in Toronto

In [10]:
# Filtering Boroughs that have the word Toronto in them
Toronto_data=Merged_df[Merged_df['Borough'].str.lower().str.find('toronto')>0]
Toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [11]:
Toronto_data.shape

(39, 5)

# Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'YT0EDB1LIXWATJQOLVZ1PSQMSN1PNAGZGZCP5SCSMYRWHMIR' # your Foursquare ID
CLIENT_SECRET = 'ZIVSV1Q1ZNU2MC1TAHBLKAXBOPATECKAUZCFVJG3F3UGZBJM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YT0EDB1LIXWATJQOLVZ1PSQMSN1PNAGZGZCP5SCSMYRWHMIR
CLIENT_SECRET:ZIVSV1Q1ZNU2MC1TAHBLKAXBOPATECKAUZCFVJG3F3UGZBJM


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(len(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #nearby_venues = pd.DataFrame([item for venue_list in venues_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Harbourfront
45
Queen's Park
42
Ryerson, Garden District
100
St. James Town
100
The Beaches
4
Berczy Park
56
Central Bay Street
86
Christie
17
King, Adelaide, Richmond
100
Dufferin, Dovercourt Village
15
Toronto Islands, Harbourfront East, Union Station
100
Little Portugal, Trinity
55
Riverdale, The Danforth West
42
Toronto Dominion Centre, Design Exchange
100
Parkdale Village, Brockton, Exhibition Place
22
India Bazaar, The Beaches West
19
Commerce Court, Victoria Hotel
100
Studio District
41
Lawrence Park
3
Roselawn
1
Davisville North
9
Forest Hill North, Forest Hill West
4
High Park, The Junction South
24
North Toronto West
21
Yorkville, North Midtown, The Annex
22
Roncesvalles, Parkdale
13
Davisville
31
University of Toronto, Harbord
38
Runnymede, Swansea
37
Moore Park, Summerhill East
3
Grange Park, Chinatown, Kensington Market
91
Summerhill West, Rathnelly, Forest Hill SE, Deer Park, South Hill
16
Railway Lands, Harbourfront West, South Niagara, CN Tower, Bathurst Quay, King and 

In [15]:
Toronto_venues['Neighborhood'].value_counts()

Commerce Court, Victoria Hotel                                                                                100
Toronto Islands, Harbourfront East, Union Station                                                             100
Ryerson, Garden District                                                                                      100
St. James Town                                                                                                100
Toronto Dominion Centre, Design Exchange                                                                      100
King, Adelaide, Richmond                                                                                      100
First Canadian Place, Underground city                                                                        100
Stn A PO Boxes 25 The Esplanade                                                                                97
Grange Park, Chinatown, Kensington Market                                               

# Analyzing each neighborhood

In [16]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
Toronto_onehot.set_index('Neighborhood',inplace=True)
Toronto_onehot.reset_index(drop=False,inplace=True)
Toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500
2,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.00,...,0.00000,0.00,0.011628,0.000000,0.000000,0.011628,0.000000,0.000000,0.00,0.011628
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.012048,0.00,0.024096
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,...,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [18]:
Toronto_grouped.shape

(39, 234)

# Cluster neighborhoods

In [19]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [20]:
# adding cluster labels
Toronto_clusters=pd.DataFrame(Toronto_grouped['Neighborhood'])
Toronto_clusters.insert(0,'Cluster Labels',kmeans.labels_)
Toronto_merged = Toronto_data
Toronto_merged=Toronto_merged.join(Toronto_clusters.set_index('Neighborhood'),on='Neighborhood')
Toronto_merged.dropna(inplace=True)
Toronto_merged.reset_index(drop=True,inplace=True)

In [21]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [23]:
colors_array

array([[5.00000000e-01, 0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.96078431e-03, 7.09281308e-01, 9.23289106e-01, 1.00000000e+00],
       [5.03921569e-01, 9.99981027e-01, 7.04925547e-01, 1.00000000e+00],
       [1.00000000e+00, 7.00543038e-01, 3.78411050e-01, 1.00000000e+00],
       [1.00000000e+00, 1.22464680e-16, 6.12323400e-17, 1.00000000e+00]])

In [28]:
colors.rgb2hex([5.0000000e-01, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00])

'#8000ff'